In [12]:
import pandas as pd

In [16]:
wmt21 = pd.read_csv("wmt_data/2021-da.csv")
wmt22 = pd.read_csv("wmt_data/2022-da.csv")
data_wmt = pd.concat([wmt21, wmt22])
data_wmt

,lp,src,mt,ref,score,raw,annotators,domain
0,zh-en,新华时评：把优秀返乡农民工打造成乡村振兴生力军-新华网,Xinhua Commentary: Making Excellent Returning ...,Xinhua Commentary: Outstanding returning rural...,-0.086212,88.0,1,news
1,zh-en,乡村振兴，人才是关键。,Talent is the key to rural revitalization.,Talent is the key to rural revitalization.,0.244415,92.0,1,news
2,zh-en,人才哪里找？,Where can I find talent?,Where is talent found?,-0.995434,77.0,1,news
3,zh-en,优秀返乡农民工群体是不可或缺的生力军。,The group of outstanding returning migrant wor...,Outstanding returning rural migrant workers ar...,0.409728,94.0,1,news
4,zh-en,新华社发 商海春 作,"Hai Chunzuo, Xinhua News Agency",Published by Xinhua News Agency Author: Shang ...,0.492384,95.0,1,news
...,...,...,...,...,...,...,...,...
112599,ru-en,Главная героиня ленты - переводчица при миссии...,The main character of the tape is a translator...,The female lead of the movie is a UN translato...,-0.077652,66.5,1,news
112600,ru-en,"Что до драмы ""Разжимая кулаки"", снятой Кирой К...","As for the drama ""Unclenching Fists,"" filmed b...","As for the drama ""Unclenching the Fists"", film...",1.142646,99.5,1,news
112601,ru-en,"Ранее было объявлено, что этот фильм номиниров...",It was previously announced that the film was ...,It was previously announced that the movie is ...,-0.144525,68.0,1,news
112602,ru-en,Как показывают регулярные дружеские встречи пр...,As the regular friendly meetings of the Presid...,As shown by the regular friendly meetings amon...,0.414980,79.0,1,social


In [17]:
data_wmt = data_wmt[data_wmt.lp.str.endswith('-en')]
data_wmt

,lp,src,mt,ref,score,raw,annotators,domain
0,zh-en,新华时评：把优秀返乡农民工打造成乡村振兴生力军-新华网,Xinhua Commentary: Making Excellent Returning ...,Xinhua Commentary: Outstanding returning rural...,-0.086212,88.0,1,news
1,zh-en,乡村振兴，人才是关键。,Talent is the key to rural revitalization.,Talent is the key to rural revitalization.,0.244415,92.0,1,news
2,zh-en,人才哪里找？,Where can I find talent?,Where is talent found?,-0.995434,77.0,1,news
3,zh-en,优秀返乡农民工群体是不可或缺的生力军。,The group of outstanding returning migrant wor...,Outstanding returning rural migrant workers ar...,0.409728,94.0,1,news
4,zh-en,新华社发 商海春 作,"Hai Chunzuo, Xinhua News Agency",Published by Xinhua News Agency Author: Shang ...,0.492384,95.0,1,news
...,...,...,...,...,...,...,...,...
112599,ru-en,Главная героиня ленты - переводчица при миссии...,The main character of the tape is a translator...,The female lead of the movie is a UN translato...,-0.077652,66.5,1,news
112600,ru-en,"Что до драмы ""Разжимая кулаки"", снятой Кирой К...","As for the drama ""Unclenching Fists,"" filmed b...","As for the drama ""Unclenching the Fists"", film...",1.142646,99.5,1,news
112601,ru-en,"Ранее было объявлено, что этот фильм номиниров...",It was previously announced that the film was ...,It was previously announced that the movie is ...,-0.144525,68.0,1,news
112602,ru-en,Как показывают регулярные дружеские встречи пр...,As the regular friendly meetings of the Presid...,As shown by the regular friendly meetings amon...,0.414980,79.0,1,social


In [34]:
from pathlib import Path
import torch
from sentence_transformers import models, losses, datasets
from sentence_transformers import LoggingHandler, SentenceTransformer, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import pandas as pd
from tqdm import tqdm
from datasets import Dataset, DatasetDict
import numpy as np
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

In [40]:
timestamp = "2023-02-17_15-02-13"
project_base_path = Path("Guided Research WS22")
negation_dataset = project_base_path / "data/negation_dataset_labeled.tsv"

base_model = "sentence-transformers/all-mpnet-base-v2"
output_model_name = f"{base_model.split('/')[1]}-negation"  # TODO.
model_save_path = str(project_base_path / f"finetuned-models/{timestamp}/{output_model_name}")
sentence_model = SentenceTransformer(model_save_path)

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)
sentence_model.to(device)

cuda:0


SentenceTransformer(
  (0): Transformer({'max_seq_length': 75, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [33]:
data_wmt = data_wmt.rename(columns={"score":"label"})
data_wmt = data_wmt.sample(frac=1).reset_index(drop=True)
df = data_wmt[["mt", "ref", "label"]]
dataset_unsplit = Dataset.from_pandas(df)  # no need to shuffle (data already shuffled)

# Split dataset into train-eval-test (80-10-10)
# HuggingFace doesn't provide a straightforward way of doing this,
# see https://discuss.huggingface.co/t/how-to-split-main-dataset-into-train-dev-test-as-datasetdict
dataset_train = dataset_unsplit.train_test_split(test_size=0.2)
dataset_eval_test = dataset_train["test"].train_test_split(test_size=0.5)

dataset = DatasetDict(
    train=dataset_train["train"],
    eval=dataset_eval_test["train"],  # not a typo!
    test=dataset_eval_test["test"]
)

print(dataset)

assert dataset_unsplit.num_rows == (dataset["train"].num_rows + dataset["eval"].num_rows + dataset["test"].num_rows)
assert dataset["train"][:10] != dataset["eval"][:10] \
       and dataset["train"][:10] != dataset["test"][:10] \
       and dataset["eval"][:10] != dataset["test"][:10]

DatasetDict({
    train: Dataset({
        features: ['mt', 'ref', 'label'],
        num_rows: 175892
    })
    eval: Dataset({
        features: ['mt', 'ref', 'label'],
        num_rows: 21986
    })
    test: Dataset({
        features: ['mt', 'ref', 'label'],
        num_rows: 21987
    })
})


In [32]:
class ScoreModel(torch.nn.Module):
    def __init__(self, sentence_model:SentenceTransformer):
        super().__init__()
        self.sent_encoder = sentence_model
        self.predictor = torch.nn.Linear(2*768, 1)

    def forward(self, candidates, references):
        cand_emd = self.sent_encoder.encode(candidates)
        ref_emb = self.sent_encoder.encode(references)
        embeddings = torch.cat([cand_emd, ref_emb])
        return self.predictor(embeddings)

model = ScoreModel(sentence_model)

In [47]:
X_train_mt = sentence_model.encode(dataset['train']['mt'])
X_train_ref = sentence_model.encode(dataset['train']['ref'])
X_train_vec = np.concatenate([X_train_mt, X_train_mt], axis=1)
print(X_train_vec.shape)

X_eval_mt = sentence_model.encode(dataset['eval']['mt'])
X_eval_ref = sentence_model.encode(dataset['eval']['ref'])
X_eval_vec = np.concatenate([X_eval_mt, X_eval_mt], axis=1)

X_test_mt = sentence_model.encode(dataset['test']['mt'])
X_test_ref = sentence_model.encode(dataset['test']['ref'])
X_test_vec = np.concatenate([X_test_mt, X_test_mt], axis=1)

(175892, 1536)


In [ ]:
clf = SVR()
clf.fit(X_train_vec, dataset['train']['label'])

labels_pred = clf.predict(X_train_vec)
print("MSE train:", mean_squared_error(dataset['train']['label'], labels_pred))

labels_pred_eval = clf.predict(X_eval_vec)
print("MSE eval:", mean_squared_error(dataset['eval']['label'], labels_pred_eval))